In [1]:
import os
import cv2
import pandas as pd
from prodigy.components.db import connect

db = connect()
db.datasets

['fall_data']

In [2]:
fall_data = db.get_dataset("fall_data")
len(db.get_dataset("fall_data"))

158

In [3]:
# db.drop_dataset("fall_data")

In [15]:
len(fall_data)

158

In [4]:
fall_data[33]['audio_spans']

[{'start': 2.1167192429,
  'end': 3.6782334385,
  'label': 'FALL',
  'id': 'dbbd7db1-5cc4-4c83-a735-b11f7010636b',
  'color': 'rgba(255,215,0,0.2)'}]

## Functions

In [5]:
def get_video_info_dict(video_path):
    cap = cv2.VideoCapture(video_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    codec = "".join([chr((fourcc >> 8 * i) & 0xFF) for i in range(4)])
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) / int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    return {'num_frames': num_frames, 'fps': fps, 'codec': codec, 'width': width, 'height': height, 'duration': duration}

## Creation of a Pandas Series for the labels

In [6]:
import re

def format_action(action_filename):
    '''
    Transform the name of an action file into the format "Subject.X/Action"

    Args:
        action_filename (str): the name of the action file
        
    Returns:
        str: the path to the action file in the format "Subject.X/Action"
    '''


    # Récupérer le numéro de sujet
    subject_number = re.search(r'S(\d+).json$', action_filename).group(1) # group(1) pour récupérer le premier groupe de la regex, qui correspond au numéro de sujet car c'est ce qui est entre parenthèses
    # Supprimer l'extension .json et le "S" suivi du numéro à la fin
    action = re.sub(r'S\d+.json$', '', action_filename)
    # Cas d'exception pour "Pickupobject"
    if action == "Pickupobject":
        formatted_action = "Pick up object"
    else:
        # Séparer les mots en utilisant les majuscules comme séparateur
        words = re.findall('[A-Z][^A-Z]*', action)
        # Mettre la première en minuscule, sauf pour le premier mot
        words = [words[0]] + [word[0].lower() + word[1:] for word in words[1:]]
        # Joindre les mots avec un espace
        formatted_action = ' '.join(words)
    return f'Subject.{subject_number}/{formatted_action}'

# Testons le script avec vos exemples
print(format_action('FallBackwardsS1.json'))  # 'Subject.1/Fall backwards'
print(format_action('HopS2.json'))  # 'Subject.2/Hop'
print(format_action('PickupobjectS3.json'))  # 'Subject.3/Pick up object'

Subject.1/Fall backwards
Subject.2/Hop
Subject.3/Pick up object


In [12]:
file_path = '../../outputs/fall/bbox_imposed/predictions/WalkS4.json'
dataset_path = '../../Data/Fall/Dataset_CAUCAFall/CAUCAFall/'
images_path = format_action(file_path.split("/")[-1])
images_list = os.listdir(dataset_path + images_path)
labels_file_list = [file for file in images_list if file.endswith(".txt")]
labels_file_list.sort()
labels_file_list.remove("classes.txt")

In [13]:
labels_list = []

for file in labels_file_list:
    with open(dataset_path + images_path + '/' + file) as f:
        class_integer = int(f.readline()[0])
        if class_integer == 0:
            labels_list.append('Normal')
        elif class_integer == 1:
            labels_list.append('Lying down')

labels_ser = pd.Series(labels_list, name='label')

labels_ser

0      Normal
1      Normal
2      Normal
3      Normal
4      Normal
        ...  
236    Normal
237    Normal
238    Normal
239    Normal
240    Normal
Name: label, Length: 241, dtype: object

In [14]:
def get_fall_label(file_path):
    for i in range(len(fall_data)):
        if fall_data[i]['text'] == file_path.split('/')[-1].split('.')[0]:
            return fall_data[i]

label_dict = get_fall_label(file_path)
label_dict

In [11]:
if 'audio_spans' in label_dict and label_dict['audio_spans']:
    print(label_dict['audio_spans'])
else:
    print('no audio spans')

no audio spans


In [ ]:
video_folder_path = '../../Data/Fall/Dataset_CAUCAFall/video/'

info_dict = get_video_info_dict(video_folder_path + label_dict['text'] + '.mp4')

In [ ]:
start_frame = label_dict['audio_spans'][0]['start'] * info_dict['fps']
end_frame = label_dict['audio_spans'][0]['end'] * info_dict['fps']
print(start_frame, end_frame)

54.98422713 90.331230284
